In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForMultipleChoice, TrainingArguments, Trainer
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch
from sklearn.metrics import accuracy_score
from transformers import EarlyStoppingCallback

In [ ]:
# Load data
train_df = pd.read_csv('/kaggle/input/super-ai-engineer-5-question-and-answer/QA/train.csv')
test_df = pd.read_csv('/kaggle/input/super-ai-engineer-5-question-and-answer/QA/test.csv')
sample_submission_df = pd.read_csv('/kaggle/input/super-ai-engineer-5-question-and-answer/QA/sample_submission.csv')

# Preprocessing
train_df = train_df.dropna()
train_df = train_df.reset_index(drop=True)

# Drop rows where the correct answer is not in ['A', 'B', 'C', 'D']
drop_indices = train_df[~train_df['correct'].isin(['A', 'B', 'C', 'D'])].index
train_df = train_df.drop(drop_indices).reset_index(drop=True)

# Split the data into training and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

# Load tokenizer
model_dir = 'microsoft/deberta-v3-large'  # Use the suggested high-performance model
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Preprocessing function
options = 'ABCD'
indices = list(range(4))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
    first_sentence = [example['question']] * 4
    second_sentence = [example[option] for option in options]
    tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
    
    # Add label only if 'correct' column exists
    if 'correct' in example:
        tokenized_example['label'] = option_to_index[example['correct']]
    
    return tokenized_example

# Tokenize datasets
tokenized_train_ds = train_ds.map(preprocess, batched=False, remove_columns=['question', 'A', 'B', 'C', 'D', 'correct'])
tokenized_val_ds = val_ds.map(preprocess, batched=False, remove_columns=['question', 'A', 'B', 'C', 'D', 'correct'])

# Data collator
@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        # Check if 'label' or 'labels' exists in the features
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) if label_name in feature else None for feature in features]
        
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        
        # Add labels to the batch only if they exist
        if any(label is not None for label in labels):
            batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        
        return batch

# Load model
model = AutoModelForMultipleChoice.from_pretrained(model_dir)

# Define compute_metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"eval_accuracy": accuracy_score(labels, predictions)}

# Training arguments with early stopping
training_args = TrainingArguments(
    output_dir=model_dir,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_accuracy',  # Must match the key in compute_metrics
    greater_is_better=True,
    learning_rate=3e-5,
    per_device_eval_batch_size=4,
    per_device_train_batch_size=4,
    num_train_epochs=4,
    weight_decay=0.01,
    report_to='none',
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=10,
    logging_strategy='steps',
    seed=42,
    fp16=True,
)

# Trainer with EarlyStoppingCallback and compute_metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_val_ds,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics,  # Add compute_metrics here
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # Updated patience to 5
)

# Train the model
trainer.train()

# Predict on validation set
val_predictions = trainer.predict(tokenized_val_ds)
val_preds = np.argmax(val_predictions.predictions, axis=1)
val_true = val_df['correct'].map(option_to_index).tolist()

# Calculate accuracy
val_accuracy = accuracy_score(val_true, val_preds)
print(f'Validation Accuracy: {val_accuracy:.4f}')

# Preprocess test dataset
test_ds = Dataset.from_pandas(test_df)
tokenized_test_ds = test_ds.map(preprocess, batched=False, remove_columns=['id', 'question', 'A', 'B', 'C', 'D'])

# Predict on test set
test_predictions = trainer.predict(tokenized_test_ds)
test_preds = np.argmax(test_predictions.predictions, axis=1)
test_preds = [index_to_option[pred] for pred in test_preds]

# Create submission DataFrame
submission_df = pd.DataFrame({'id': test_df['id'], 'answer': test_preds})
submission_df.to_csv('/kaggle/working/submission_question_answer.csv', index=False)
submission_df.head(3)